<a href="https://colab.research.google.com/github/aelukin/ML_ITMO/blob/main/watch_market_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Монтируем диск, задаем директорию

In [640]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [641]:
%cd gdrive/MyDrive/ML\ ITMO

[Errno 2] No such file or directory: 'gdrive/MyDrive/ML ITMO'
/content/gdrive/MyDrive/ML ITMO


Импортируем библиотеки

In [642]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Импортируем датасет

In [643]:
path='Watches.csv'
data=pd.read_csv(path, index_col='Unnamed: 0')

<ipython-input-643-50ca43d5d0b5>:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(path, index_col='Unnamed: 0')


# Часть 1. Проверка и очистка данных

In [644]:
data.head()

,name,price,brand,model,ref,mvmt,casem,bracem,yop,cond,sex,size,condition
0,Audemars Piguet Royal Oak Offshore Chronograph...,"$43,500",Audemars Piguet,Royal Oak Offshore Chronograph,26237ST.OO.1000ST.01,NaN,NaN,NaN,2019,Unworn,Men's watch/Unisex,42 mm,NaN
1,Audemars Piguet Royal Oak Selfwinding\n39mm Bl...,"$71,500",Audemars Piguet,Royal Oak Selfwinding,15300ST.OO.1220ST.02,NaN,NaN,NaN,2012,Very good,Men's watch/Unisex,39 mm,NaN
2,Audemars Piguet Royal Oak Chronograph\nBlue Di...,"$79,191",Audemars Piguet,Royal Oak Chronograph,26331ST,Automatic,Steel,Steel,Unknown,Unworn,NaN,41 mm,NaN
3,Audemars Piguet Royal Oak Chronograph\nSelfwin...,"$108,000",Audemars Piguet,Royal Oak Chronograph,26715ST.OO.1356ST.01,Automatic,Steel,Steel,2022 (Approximation),New,Men's watch/Unisex,38 mm,NaN
4,Audemars Piguet Royal Oak Offshore Chronograph...,"$27,500",Audemars Piguet,Royal Oak Offshore Chronograph,26170ST.OO.1000ST.01,Automatic,Steel,Steel,Unknown,Very good,Men's watch/Unisex,42 x 54 mm,NaN


In [645]:
data.tail()

,name,price,brand,model,ref,mvmt,casem,bracem,yop,cond,sex,size,condition
6438,Zenith Defy El Primero\n21 TITANIUM 95.9000.90...,"$9,790",Zenith,Defy El Primero,95.9000.9004/78.M9000,NaN,NaN,NaN,2022,Very good,Men's watch/Unisex,44 mm,NaN
6439,Zenith Chronomaster Sport\n41mm White 03.3100....,"$8,450",Zenith,Chronomaster Sport,03.3100.3600/69.M3100,NaN,NaN,NaN,2021 (Approximation),Very good,Men's watch/Unisex,41 mm,NaN
6440,Zenith El Primero\n50th Anniversary A386 Limited,"$16,500",Zenith,El Primero,30.A386.400/69.C807,NaN,NaN,NaN,2019,Very good,Men's watch/Unisex,38 mm,NaN
6441,Zenith Chronomaster Sport\nWhite Dial Chronogr...,"$9,000",Zenith,Chronomaster Sport,03.3100.3600/69.M3100,NaN,NaN,NaN,2021,Unworn,Men's watch/Unisex,41 mm,NaN
6442,Zenith El Primero Chronomaster\n03.2040.400/69...,"$6,833",Zenith,El Primero Chronomaster,NaN,Automatic,Steel,Leather,2019,Very good,NaN,42 mm,NaN


Проверка данных

In [646]:
data.columns

Index(['name', 'price', 'brand', 'model', 'ref', 'mvmt', 'casem', 'bracem',
       'yop', 'cond', 'sex', 'size', 'condition'],
      dtype='object')

In [647]:
data.shape

(284491, 13)

In [648]:
data.dtypes

name         object
price        object
brand        object
model        object
ref          object
mvmt         object
casem        object
bracem       object
yop          object
cond         object
sex          object
size         object
condition    object
dtype: object

In [649]:
data.isna().sum()

name          72585
price           406
brand           131
model         30466
ref           43152
mvmt         196685
casem        164271
bracem       174896
yop             134
cond          75987
sex           95805
size          23597
condition    212922
dtype: int64

In [650]:
data.describe()

,name,price,brand,model,ref,mvmt,casem,bracem,yop,cond,sex,size,condition
count,211906,284085,284360,254025,241339,87806,120220,109595,284357,208504,188686,260894,71569
unique,148689,36830,28,948,33223,3,17,23,410,7,2,6699,7
top,Omega Speedmaster Professional Moonwatch\nCo-A...,Price on request,Rolex,Datejust 36,126334,Automatic,Steel,Steel,Unknown,Very good,Men's watch/Unisex,40 mm,Very good
freq,171,14259,72484,9473,2081,65600,75793,49605,95957,71604,158841,34254,36136


In [651]:
data['brand'].unique()

array(['Audemars Piguet', 'Cartier', 'Tudor', nan, 'Breitling', 'Oris',
       'Richard Mille', 'Hublot', 'IWC', 'Jaeger-LeCoultre', 'Longines',
       'Omega', 'Panerai', 'Patek Philippe', 'Rolex', 'Seiko', 'Sinn',
       'Edox', 'NOMOS', 'Zenith', 'Tissot', 'Hamilton', 'Meistersinger',
       'Rado', 'Montblanc', 'TAG Heuer', 'Ebel', 'A. Lange & Söhne',
       'Vacheron Constantin'], dtype=object)

In [652]:
data['mvmt'].unique()

array([nan, 'Automatic', 'Manual winding', 'Quartz'], dtype=object)

In [653]:
data['casem'].unique()

array([nan, 'Steel', 'Titanium', 'White gold', 'Ceramic', 'Rose gold',
       'Yellow gold', 'Gold/Steel', 'Platinum', 'Carbon', 'Red gold',
       'Tantalum', 'Aluminum', 'Bronze', 'Plastic', 'Silver', 'Palladium',
       'Tungsten'], dtype=object)

In [654]:
data['bracem'].unique()

array([nan, 'Steel', 'Rubber', 'Crocodile skin', 'White gold', 'Ceramic',
       'Rose gold', 'Yellow gold', 'Titanium', 'Calf skin', 'Gold/Steel',
       'Leather', 'Platinum', 'Satin', 'Textile', 'Red gold',
       'Lizard skin', 'Plastic', 'Aluminium', 'Silicon', 'Shark skin',
       'Silver', 'Snake skin', 'Ostrich skin'], dtype=object)

In [655]:
data['cond'].unique()

array(['Unworn', 'Very good', 'New', 'Good', 'Fair', nan, 'Poor',
       'Incomplete'], dtype=object)

In [656]:
data['sex'].unique()

array(["Men's watch/Unisex", nan, "Women's watch"], dtype=object)

In [657]:
data['size'].unique()

array(['42 mm', '39 mm', '41 mm', ..., '40 x 10.75 mm', '46 x 17 mm',
       '48 x 57.5 mm'], dtype=object)

In [658]:
data['condition'].unique()

array([nan, 'Very good', 'Unworn', 'Good', 'Fair', 'New', 'Poor',
       'Incomplete'], dtype=object)

Ищем и удаляем дублирующиеся ячейки

In [659]:
data.isna().sum()

name          72585
price           406
brand           131
model         30466
ref           43152
mvmt         196685
casem        164271
bracem       174896
yop             134
cond          75987
sex           95805
size          23597
condition    212922
dtype: int64

In [660]:
data.duplicated().sum()

11398

In [661]:
data=data.drop_duplicates()

In [662]:
data.describe()

,name,price,brand,model,ref,mvmt,casem,bracem,yop,cond,sex,size,condition
count,206862,272783,273028,243114,230967,84667,113932,104257,273025,203593,184005,250359,65258
unique,148689,36830,28,948,33223,3,17,23,410,7,2,6699,7
top,Omega Speedmaster Professional Moonwatch\nCo-A...,Price on request,Rolex,Datejust 36,126334,Automatic,Steel,Steel,Unknown,Very good,Men's watch/Unisex,40 mm,Very good
freq,169,11934,66130,8357,1731,63144,72373,47387,91387,71007,155235,32650,33267


In [663]:
data['condition'].notna().sum()+data['cond'].notna().sum()

268851

Идентифицированные проблемы датасета:

*   Колонки 'cond' and 'condition'. Есть основания полагать, что по какой-то причине значения "состояния" разбиты на две колонки, которые необходимо объединить вручную.
*   Цена 'price' в текстовом формате
*   Лишняя информация в столбце 'size'
*   Столбец с годом производства 'yop' требует дополнительной проверки







1. Исправляем проблему с cond и condition


In [664]:
data['cond'] = data['cond'].fillna('')
data['condition'] = data['condition'].fillna('')
data['condition_summed']=data['cond'].astype(str)+data['condition'].astype(str)

In [665]:
data.describe()

,name,price,brand,model,ref,mvmt,casem,bracem,yop,cond,sex,size,condition,condition_summed
count,206862,272783,273028,243114,230967,84667,113932,104257,273025,273093,184005,250359,273093,273093
unique,148689,36830,28,948,33223,3,17,23,410,8,2,6699,8,8
top,Omega Speedmaster Professional Moonwatch\nCo-A...,Price on request,Rolex,Datejust 36,126334,Automatic,Steel,Steel,Unknown,Very good,Men's watch/Unisex,40 mm,,Very good
freq,169,11934,66130,8357,1731,63144,72373,47387,91387,71007,155235,32650,207835,104274


In [666]:
data['cond'].replace('', np.nan, inplace=True)
data['condition'].replace('', np.nan, inplace=True)
data['condition_summed'].replace('', np.nan, inplace=True)

In [667]:
data.condition.value_counts()

Very good     33267
Unworn        12453
New           10186
Good           8004
Fair           1300
Poor             45
Incomplete        3
Name: condition, dtype: int64

In [668]:
data.cond.value_counts()

Very good     71007
New           65565
Unworn        32182
Good          27203
Fair           7446
Poor            148
Incomplete       42
Name: cond, dtype: int64

In [669]:
data.condition_summed.value_counts().sum()

268851

Число значений в модифицированном столбце совпало с расчетным. Теперь можно убрать столбцы cond и condition

In [670]:
data=data.drop(columns=['cond', 'condition'])
data.describe()

,name,price,brand,model,ref,mvmt,casem,bracem,yop,sex,size,condition_summed
count,206862,272783,273028,243114,230967,84667,113932,104257,273025,184005,250359,268851
unique,148689,36830,28,948,33223,3,17,23,410,2,6699,7
top,Omega Speedmaster Professional Moonwatch\nCo-A...,Price on request,Rolex,Datejust 36,126334,Automatic,Steel,Steel,Unknown,Men's watch/Unisex,40 mm,Very good
freq,169,11934,66130,8357,1731,63144,72373,47387,91387,155235,32650,104274


2. Форматируем столбец "Price"

убираем спецсимволы и не-числовые данные

In [671]:
data['price']=data['price'].str.replace('\W','',regex=True)
data['price']=data['price'].str.replace('(\D+)', '',regex=True)
data['price'].replace('', np.nan, inplace=True)

удаляем ячейки с NaN

In [672]:
data=data.dropna(subset=['price'])

Конвертируем столбец в int

In [673]:
data['price']=data['price'].astype(int)

<ipython-input-673-2a10b3d39cc2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price']=data['price'].astype(int)


In [674]:
data.dtypes

name                object
price                int64
brand               object
model               object
ref                 object
mvmt                object
casem               object
bracem              object
yop                 object
sex                 object
size                object
condition_summed    object
dtype: object

In [675]:
data.shape

(260849, 12)

In [676]:
data.price.value_counts()

6500      249
4500      239
12500     236
4026      236
8500      225
         ... 
126802      1
88286       1
289294      1
64993       1
45998       1
Name: price, Length: 36829, dtype: int64

3. Форматируем столбец **size**

In [677]:
data['size'].unique()

array(['42 mm', '39 mm', '41 mm', ..., '40 x 10.75 mm', '46 x 17 mm',
       '48 x 57.5 mm'], dtype=object)

In [678]:
data['size'].value_counts()

40 mm             31119
41 mm             25136
42 mm             23198
36 mm             17834
44 mm             13274
                  ...  
34.5 x 35 mm          1
38.6 x 38,6 mm        1
31.5 x 38 mm          1
26.8 x 32.5 mm        1
48 x 57.5 mm          1
Name: size, Length: 6595, dtype: int64

Информация о диаметре корпуса содержится в первом числе. Вторым числом является либо толщина, либо дополнительное измерение корпуса (в случае часов квадратной/овальной формы). Оставляем только диаметр.

In [679]:
data['size']=data['size'].str[0:2]

In [680]:
data['size'].unique()

array(['42', '39', '41', '38', '44', '37', nan, '34', '36', '45', '43',
       '23', '33', '24', '40', '48', '31', '29', '35', '22', '26', '25',
       '90', '65', '15', '30', '28', '32', '47', 'Re', 'Br', '64', '14',
       'Au', '20', '50', '49', '18', '51', '27', '16', '68', '21', '46',
       '61', 'Li', 'Ma', '69', '70', '55', '54', '17', '[P', 'Th', 'Mo',
       '57', '-M', '8 ', '8S', 'Pr', '19', '59', '56', 'I ', '53', '52',
       'De', 'Yo', 'Su', '13', 'Of', '58', '"A', 'Ai', '【a', '72', '84',
       'Or', '11', '3 ', 'Fo', '75', '12', '85', '6.', 'XL', 'ma', '78',
       '63', 'Ca', '73', 'St', '79', '60', '10', '7.', '"R', 'Am', 'In',
       '"P', 'Pa', 'Ba', 'Wa', 'A ', '7,', '"C', 'It', '"T', 'Sc', 'Sa',
       '[S', '*B', '"M', '"E', '"N', '【r', 'SP', 'IT', 'La', 'Mi', '9 ',
       '5.', '6 ', 'Lo', 'Hu', '※O', '91', 'Ce', 'Un', 'IW', 'iw', '*D',
       '*S', '5 ', '[B', 'ca', '[N', '4 ', '80', 'Vi', '2 ', 'Ne', '96',
       'Ja', '76', '66', 'On', 'JL', 'WE', '1.', '3.

In [681]:
data['size']=data['size'].str.replace('\W','',regex=True)
data['size']=data['size'].str.replace('(\D+)', '',regex=True)
data['size'].replace('', np.nan, inplace=True)

In [682]:
data['size'].unique()

array(['42', '39', '41', '38', '44', '37', nan, '34', '36', '45', '43',
       '23', '33', '24', '40', '48', '31', '29', '35', '22', '26', '25',
       '90', '65', '15', '30', '28', '32', '47', '64', '14', '20', '50',
       '49', '18', '51', '27', '16', '68', '21', '46', '61', '69', '70',
       '55', '54', '17', '57', '8', '19', '59', '56', '53', '52', '13',
       '58', '72', '84', '11', '3', '75', '12', '85', '6', '78', '63',
       '73', '79', '60', '10', '7', '9', '5', '91', '4', '80', '2', '96',
       '76', '66', '1', '67', '71', '0', '３４', '93', '83', '62'],
      dtype=object)